In [1]:
import pandas as pd
import re

In [26]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

,index_df,x,y,data-hora,data,dia,mes,ano,tempo,hora,...,c,d,e,f,placa,h,i,j,k,l
0,1,501,1,2019-02-01 00:00:07,2019-02-01,1,2,2019,00:00:07,0,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
1,2,502,2,2019-02-01 00:00:09,2019-02-01,1,2,2019,00:00:09,0,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN
2,3,502,2,2019-02-01 00:00:17,2019-02-01,1,2,2019,00:00:17,0,...,61.0,2,3.6,bdac46ee2,LQM9654,0,0,NaN,NaN,NaN
3,4,501,1,2019-02-01 00:00:20,2019-02-01,1,2,2019,00:00:20,0,...,73.0,2,3.8,bdac470d1,LMR8029,0,0,NaN,NaN,NaN


In [29]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

# Correção para linhas que começam com múltiplos espaços
def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)
            
# Separando as colunas manualmente
df_ocr['x'] = df_ocr.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_ocr['x'] = df_ocr.x.apply(lambda x: correctSplit(x))
df_ocr['placa'] = df_ocr.x.apply(lambda x: x.split('|')[0])
df_ocr['b'] = df_ocr.x.apply(lambda x: x.split('|')[1])
df_ocr['c'] = df_ocr.x.apply(lambda x: x.split('|')[2])
df_ocr['bairro'] = df_ocr.x.apply(lambda x: x.split('|')[3])
df_ocr['e'] = df_ocr.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_ocr.drop('x', axis = 1, inplace = True)

df_ocr.head()

,placa,b,c,bairro,e
0,KPV68771AUTOMÓVEL,20190101000000971024,0101201900000003900CET8-5455 0410191112025201...,St R Catete,B-22.93380-43.18294
1,LQR82471AUTOMÓVEL,20190101000001181044,0101201900000103400CET8-5410 0410121122025201...,St Tanque,D-22.97019-43.41377
2,ETP18901AUTOMÓVEL,20190101000001281014,0101201900000104000CET8-5436 0410241111025201...,St Cidade Nova,A-22.93149-43.20888
3,KXL99161AUTOMÓVEL,20190101000001381014,0101201900000102500CET8-5439 0410161111025201...,St Colonia,A-22.92479-43.38907
4,LPU22871AUTOMÓVEL,20190101000001901024,0101201900000104400CET8-5401 0410381122025201...,St Pontal,B-23.01267-43.38900
